In [37]:
using Pkg
Pkg.activate("/media/mat/HDD/EnKF/")

"/media/mat/HDD/EnKF/Project.toml"

In [38]:
using Revise
using EnKF
using Distributions
using LinearAlgebra

In [39]:
f = PropagationFunction()
m = MeasurementFunction()
z = RealMeasurementFunction()

RealMeasurementFunction()

In [40]:
function (::PropagationFunction)(t::Float64, ENS::EnsembleState{N, NS, TS}) where {N, NS, TS}
    return ENS  
end

In [41]:
function (::MeasurementFunction)(t::Float64, mENS::EnsembleState{N, NZ, TZ},  ENS::EnsembleState{N, NS, TS}) where {N, NS, TS, NZ, TZ}
    fill!(mENS, 1.01*ones(NZ))
    return mENS
end

In [42]:
function (::MeasurementFunction)(t::Float64, S::Array{TS,1}) where {TS}
    return 1.05*ones(NZ)
end

In [43]:
function (::RealMeasurementFunction)(t::Float64, zENS::EnsembleState{N, NZ, TZ}) where {N, NZ, TZ} 
    fill!(zENS, 1.01*ones(NZ))
    return zENS
end

#### Ensemble Kalman filter

In [44]:
N = 10
NS = 3
TS = Float64
NZ = 5
TZ = Float64
β = MultiplicativeInflation(NS, 1.01)
ϵ = AdditiveInflation(NZ)
isinflated = true
isaugmented = false

false

In [66]:
enkf = ENKF{N, TS, NZ, TZ}(f, β, m, z, ϵ, isinflated, isaugmented)

ENKF{10,Float64,5,Float64}(PropagationFunction(), MultiplicativeInflation{3}(1.01), MeasurementFunction(), RealMeasurementFunction(), AdditiveInflation{5}(IsoNormal(
dim: 5
μ: [0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)
), true, false)

In [67]:
ens = EnsembleState(N, NS)

EnsembleState{10,3,Float64}(Array{Float64,1}[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

#### Initialize each ensemble member

In [68]:
initialize(ens)

EnsembleState{10,3,Float64}(Array{Float64,1}[[-1.66471, 0.676053, -1.22097], [-0.0266453, 2.29579, -0.686862], [0.138188, -0.505176, 0.193189], [0.200068, -2.16938, -0.319361], [-0.531064, 1.50317, 0.510158], [1.06139, 1.14497, 1.85255], [0.318791, -0.871691, -0.961866], [-0.0828105, -0.133427, 0.211076], [1.61859, 0.80286, 0.523287], [-1.81119, 0.951695, 0.684395]])

#### Propagate each ensemble member

In [69]:
enkf.f(0.0, ens)

EnsembleState{10,3,Float64}(Array{Float64,1}[[-1.66471, 0.676053, -1.22097], [-0.0266453, 2.29579, -0.686862], [0.138188, -0.505176, 0.193189], [0.200068, -2.16938, -0.319361], [-0.531064, 1.50317, 0.510158], [1.06139, 1.14497, 1.85255], [0.318791, -0.871691, -0.961866], [-0.0828105, -0.133427, 0.211076], [1.61859, 0.80286, 0.523287], [-1.81119, 0.951695, 0.684395]])

#### Apply convariance inflation

In [70]:
if isinflated ==true
        enkf.A(ens)
end

EnsembleState{10,3,Float64}(Array{Float64,1}[[-1.68058, 0.679119, -1.23396], [-0.0261324, 2.31505, -0.694516], [0.140349, -0.513923, 0.194335], [0.202848, -2.19476, -0.323341], [-0.535595, 1.5145, 0.514474], [1.07279, 1.15272, 1.87029], [0.322758, -0.884103, -0.97227], [-0.0828592, -0.138456, 0.212402], [1.63555, 0.807194, 0.527735], [-1.82852, 0.957517, 0.690453]])

In [71]:
ens

EnsembleState{10,3,Float64}(Array{Float64,1}[[-1.68058, 0.679119, -1.23396], [-0.0261324, 2.31505, -0.694516], [0.140349, -0.513923, 0.194335], [0.202848, -2.19476, -0.323341], [-0.535595, 1.5145, 0.514474], [1.07279, 1.15272, 1.87029], [0.322758, -0.884103, -0.97227], [-0.0828592, -0.138456, 0.212402], [1.63555, 0.807194, 0.527735], [-1.82852, 0.957517, 0.690453]])

#### Compute mean and deviation

In [72]:
Ŝ = deepcopy(mean(ens))

3-element Array{Float64,1}:
 -0.07793972916534782
  0.3694865307017862 
  0.07855973997475506

In [73]:
ensfluc = EnsembleState(size(ens))

deviation(ensfluc, ens)
A′ = hcat(ensfluc)

3×10 Array{Float64,2}:
 -1.60264    0.0518074   0.218289  …  -0.00491949  1.71349   -1.75058 
  0.309632   1.94557    -0.88341      -0.507943    0.437707   0.58803 
 -1.31252   -0.773076    0.115775      0.133842    0.449175   0.611893

#### Compute measurement

In [74]:
mens = EnsembleState((N, NZ))
enkf.m(0.0, mens, ens)
Â = hcat(deepcopy(mens))

5×10 Array{Float64,2}:
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01

#### Compute deviation from measurement of the mean

In [75]:
Â′  = Â .- enkf.m(0.0, mean(ens))

5×10 Array{Float64,2}:
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04

#### Get actual measurements

In [76]:
zens = EnsembleState((N, NZ))
M = 1.01*ones(NZ)
enkf.z(0.0, zens)

EnsembleState{10,5,Float64}(Array{Float64,1}[[1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01]])

In [77]:
zens

EnsembleState{10,5,Float64}(Array{Float64,1}[[1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01]])

#### Perturb actual measurement

In [78]:
ϵ(zens)

EnsembleState{10,5,Float64}(Array{Float64,1}[[0.171374, 0.0713985, 1.46292, 0.528676, 1.38181], [1.20128, 0.0604121, 1.58649, 0.874224, 1.43439], [3.2031, 0.640649, 0.524665, 0.642677, 1.65493], [0.230927, 2.02628, 0.641645, 0.951238, 0.726476], [1.49078, 0.487333, 0.476496, 0.261838, 0.285854], [-0.0252117, 0.48012, -0.143654, 0.134917, 0.709577], [-0.00502122, -0.238678, 0.47087, 0.0564351, 1.74913], [1.35316, 2.22454, 1.64712, 0.887743, 1.52322], [1.49455, 1.3144, 1.23636, 1.35456, -0.972444], [0.26295, 1.68819, -0.585303, 1.77463, 1.60714]])

#### Construct representer matrix and vectors

In [79]:
D = hcat(zens)

5×10 Array{Float64,2}:
 0.171374   1.20128    3.2031    0.230927  …  1.35316    1.49455    0.26295 
 0.0713985  0.0604121  0.640649  2.02628      2.22454    1.3144     1.68819 
 1.46292    1.58649    0.524665  0.641645     1.64712    1.23636   -0.585303
 0.528676   0.874224   0.642677  0.951238     0.887743   1.35456    1.77463 
 1.38181    1.43439    1.65493   0.726476     1.52322   -0.972444   1.60714 

In [80]:
cov(ϵ)

5×5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0

In [81]:
(Â′*Â′') + (N-1)*cov(ϵ)*I

5×5 Array{Float64,2}:
 9.016  0.016  0.016  0.016  0.016
 0.016  9.016  0.016  0.016  0.016
 0.016  0.016  9.016  0.016  0.016
 0.016  0.016  0.016  9.016  0.016
 0.016  0.016  0.016  0.016  9.016

In [82]:
@time b = ((Â′*Â′') + (N-1)*cov(ϵ)*I) \ (D - Â)

  0.817178 seconds (3.41 M allocations: 162.344 MiB, 5.45% gc time)


5×10 Array{Float64,2}:
 -0.0928999   0.0212323   0.243362   …   0.0376229   0.0539612  -0.0829464
 -0.104008   -0.105531   -0.0413554      0.134443    0.0339437   0.0754138
  0.0506054   0.0640331  -0.0542425      0.0702851   0.0252735  -0.177197 
 -0.0531997  -0.0151072  -0.0411301     -0.0140904   0.0384059   0.0850181
  0.0415925   0.047134    0.071342       0.0565179  -0.22015     0.0664076

In [83]:
@time Bᵀb = (A′*Â′')*b

  0.003094 seconds (4.05 k allocations: 225.549 KiB)


3×10 Array{Float64,2}:
 -7.92192e-19   5.90044e-20   8.92857e-19  …  -3.43974e-19  -1.67075e-19
  1.08555e-18  -8.08542e-20  -1.22349e-18      4.71351e-19   2.28944e-19
 -1.51988e-18   1.13205e-19   1.71302e-18     -6.59941e-19  -3.20547e-19

#### Analysis step

In [84]:
ens  += cut(Bᵀb)

EnsembleState{10,3,Float64}(Array{Float64,1}[[-1.68058, 0.679119, -1.23396], [-0.0261324, 2.31505, -0.694516], [0.140349, -0.513923, 0.194335], [0.202848, -2.19476, -0.323341], [-0.535595, 1.5145, 0.514474], [1.07279, 1.15272, 1.87029], [0.322758, -0.884103, -0.97227], [-0.0828592, -0.138456, 0.212402], [1.63555, 0.807194, 0.527735], [-1.82852, 0.957517, 0.690453]])

In [112]:
TP = TupleProduct((Normal(), TruncatedNormal(0.0,1.0,-0.1,0.1)))

TupleProduct{2,Continuous,Tuple{Normal{Float64},Truncated{Normal{Float64},Continuous}}}(
v: (Normal{Float64}(μ=0.0, σ=1.0), Truncated(Normal{Float64}(μ=0.0, σ=1.0), range=(-0.1, 0.1)))
)


In [111]:
typeof(TP)<:MultivariateDistribution

true